In [6]:
import os
import urllib.request
import zipfile
from pathlib import Path

In [7]:
url = "https://github.com/Shenggan/BCCD_Dataset/archive/refs/heads/master.zip"
filename = "bccd.zip"
extract_dir = "bccd_dataset"
dataset_folder_name = "BCCD_Dataset-master"

In [3]:
urllib.request.urlretrieve(url, filename)

('bccd.zip', <http.client.HTTPMessage at 0x1dc8d7880e0>)

In [4]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(dir)

In [8]:
bccd_root = os.path.join(extract_dir, dataset_folder_name)

In [ ]:
os.listdir(dir)

['BCCD_Dataset-master']